Tests for retrieving data from PACS
--

Imports
--

In [55]:
import logging
import configparser
import pandas as pd
from collections import namedtuple

from datetime import datetime as dt

from IPython.core import display as ICD

from pydicom.dataset import Dataset

from scripts.run_all import run_all
from scripts.retrieve_data_from_PACS import *

#from pynetdicom import debug_logger
#debug_logger()

pd.set_option('display.max_colwidth', -1)

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Set the logger of the pynetdicom module to ERROR, to avoid logs

In [56]:
logging.getLogger('pynetdicom').setLevel(logging.ERROR)

Initialize the "config" object
--

In [57]:
config = run_all()

2019-10-24 16:28:15,033|INFO| Reading configuration
2019-10-24 16:28:15,041|INFO| Starting SchedVisu workflow
2019-10-24 16:28:15,042|INFO| Finished running SchedVisu workflow


Get all PT studies for a day
--

In [62]:
df_all_studies = find_PT_studies_for_day(config, '20190806')
df_all_studies

2019-10-24 16:29:09,614|INFO| Retrieving all studies from PACS for day 20190806


,Study Date,Study Time,Modalities in Study,Referring Physician's Name,Study Description,Patient ID,Study Instance UID
0,20190806,080313,"['PT', 'OT', 'NM', 'CT', 'SR']",DR CARDIOLOGIE^.,PET^2_Pet_Rb82_Coeur (Adult),81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0
1,20190806,084230,"['PT', 'OT', 'NM', 'CT', 'SR']",DR AMBULATOIRE^.,PET^2_Pet_Rb82_Coeur (Adult),246235,1.2.826.0.1.3680043.2.146.2.20.246235.1900153607.0
2,20190806,084334,"['PT', 'NM', 'CT', 'SR']",DIR. DEP. DE RADIOLOGIE MEDICALE^.,HEPATOBILIAIRE Q,1052884,1.2.826.0.1.3680043.2.146.2.20.1052884.1900183988.0
3,20190806,085145,"['PT', 'CT', 'SR']",DR ONCOLOGIE MEDICALE^.,PET-CT FDG WB,2993332,1.2.826.0.1.3680043.2.146.2.20.2993332.1900115202.0
4,20190806,092544,"['PT', 'CT', 'SR']",WAGNER^ANNA DOROTHEA,PET^1_Pet_FDG_CorpsEntier_Flow_ONCOFREEZE_3min (Adult),3277152,1.2.826.0.1.3680043.2.146.2.20.3277152.1900185635.0
5,20190806,093146,"['PT', 'CT', 'SR']",DR ONCOLOGIE MEDICALE^.,PET-CT FDG TRONC,3119328,1.2.826.0.1.3680043.2.146.2.20.3119328.1900121510.0
6,20190806,100712,"['PT', 'CT', 'SR']",DR ONCOLOGIE MEDICALE^.,PET-CT FDG WB,3153669,1.2.826.0.1.3680043.2.146.2.20.3153669.1900122277.0
7,20190806,101342,"['PT', 'CT', 'SR']",DR ONCOLOGIE MEDICALE^.,PET^1_Pet_FDG_Tronc_Flow (Adult),673780,1.2.826.0.1.3680043.2.146.2.20.673780.1900128678.0
8,20190806,102200,"['PT', 'CT', 'SR']",,PET-CT INJECTION,3256241,1.2.250.1.38.2.1.14.1.41104.225586
9,20190806,103301,"['PT', 'NM', 'CT', 'SR', 'PR']",IFTICENE TREBOUX^ASSIA,Os_1Phase,262905,1.2.826.0.1.3680043.2.146.2.20.262905.1900130862.0


Get all PT and CT series for the found studies
--

In [45]:
series_level_filters = ['Study Date', 'Patient ID']
to_drop_columns = ['Query/Retrieve Level', 'Retrieve AE Title', 'Type of Patient ID', 'Issuer of Patient ID']
sort_columns = ['Series Time', 'Number of Series Related Instances']

#for i_study in range(len(df_all_studies)):
for i_study in range(1):
    logging.info('DataFrame row:\n' + str(df_all_studies.loc[i_study, :]))
    
    # create the query dataset
    query_ds = create_dataset_from_dataframe_row(df_all_studies.loc[i_study, :], 'SERIES', incl=series_level_filters)
    # parameters for filtering
    query_ds.SeriesDate = query_ds.StudyDate
    query_ds.Modality = ['PT', 'CT']
    # parameters to fetch
    query_ds.SeriesInstanceUID = ''
    query_ds.StudyInstanceUID = ''
    query_ds.SeriesTime = ''
    query_ds.StudyTime  = ''
    query_ds.NumberOfSeriesRelatedInstances = ''
    query_ds.SeriesDescription = ''
    # display the query dataset
    logging.info('Query Dataset:')
    for s in str(query_ds).split('\n'): logging.info('    ' + s)
    
    # do the query (C-FIND)
    df_series = find_data(config, query_ds)
    
    # drop unwanted columns, sort and display
    df_series = df_series.drop(to_drop_columns, axis=1)
    df_series.sort_values(sort_columns,inplace=True)
    df_series.reset_index(drop=True,inplace=True)
    ICD.display(df_series)

2019-10-24 16:22:04,568|INFO| DataFrame row:
Study Date                    20190806                                         
Study Time                    080313                                           
Modalities in Study           ['PT', 'OT', 'NM', 'CT', 'SR']                   
Referring Physician's Name    DR CARDIOLOGIE^.                                 
Study Description             PET^2_Pet_Rb82_Coeur (Adult)                     
Patient ID                    81334                                            
Study Instance UID            1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0
Name: 0, dtype: object
2019-10-24 16:22:04,570|INFO| Query Dataset:
2019-10-24 16:22:04,571|INFO|     (0008, 0020) Study Date                          DA: '20190806'
2019-10-24 16:22:04,572|INFO|     (0008, 0021) Series Date                         DA: '20190806'
2019-10-24 16:22:04,573|INFO|     (0008, 0030) Study Time                          TM: ''
2019-10-24 16:22:04,573|INFO|     (0008, 

,Study Date,Series Date,Study Time,Series Time,Modality,Series Description,Patient ID,Study Instance UID,Series Instance UID,Number of Series Related Instances
0,20190806,20190806,080313,080922,CT,Topogram 0.6 T80f,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080605520646600000009,1
1,20190806,20190806,080313,081033,CT,AC CT Cardiac 3.0 HD_FoV rest iMAR,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607103330600000000,132
2,20190806,20190806,080313,081242,PT,Rest PET CardioFreeze single 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600014406,1056
3,20190806,20190806,080313,081242,PT,Rest PET CardioFreeze dual 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600018501,1056
4,20190806,20190806,080313,081242,PT,Rest PET Cardiac gated 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600016387,1056
5,20190806,20190806,080313,081242,PT,Rest PET Cardiac Static,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600000664,132
6,20190806,20190806,080313,081242,PT,Rest PET Cardiac NAC Static,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600000265,132
7,20190806,20190806,080313,081242,PT,Rest PET Cardiac Dynamic,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600006081,2772
8,20190806,20190806,080313,082112,PT,PET Statistics,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607211226300000001,18
9,20190806,20190806,080313,083024,PT,Stress PET CardioFreezeDual 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600037001,1056


Filter out some Series that do not add value (not AcquisitionTime or such)
--

In [46]:
series_descr_to_exclude = ['PET Statistics', 'Patient Protocol', 'PET Dose Report']
df_series = df_series[~df_series['Series Description'].isin(series_descr_to_exclude)].reset_index(drop=True)
df_series

,Study Date,Series Date,Study Time,Series Time,Modality,Series Description,Patient ID,Study Instance UID,Series Instance UID,Number of Series Related Instances
0,20190806,20190806,080313,080922,CT,Topogram 0.6 T80f,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080605520646600000009,1
1,20190806,20190806,080313,081033,CT,AC CT Cardiac 3.0 HD_FoV rest iMAR,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607103330600000000,132
2,20190806,20190806,080313,081242,PT,Rest PET CardioFreeze single 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600014406,1056
3,20190806,20190806,080313,081242,PT,Rest PET CardioFreeze dual 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600018501,1056
4,20190806,20190806,080313,081242,PT,Rest PET Cardiac gated 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600016387,1056
5,20190806,20190806,080313,081242,PT,Rest PET Cardiac Static,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600000664,132
6,20190806,20190806,080313,081242,PT,Rest PET Cardiac NAC Static,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600000265,132
7,20190806,20190806,080313,081242,PT,Rest PET Cardiac Dynamic,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600006081,2772
8,20190806,20190806,080313,083024,PT,Stress PET CardioFreezeDual 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600037001,1056
9,20190806,20190806,080313,083024,PT,Stress PET Cardio Gated 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600034888,1056


Go through each Series and get the first and last slice
--

In [47]:
# create a copy of the Dataframe to be updated with the fetched values
df_series = df_series.copy()

image_level_filters = ['Patient ID', 'Study Date', 'Series Instance UID', 'Modality']
to_fetch_params = ['StudyDescription', 'PatientID', 'InstitutionName', '0x07a31023', 'ReferringPhysicianName',
                        'SOPClassUID', 'ManufacturerModelName', 'InstanceNumber', 'ContentTime',
                        'AcquisitionTime', 'ActualFrameDuration', 'NumberOfSlices']

# go through each Serie
for i_serie in range(len(df_series)):
    
    logging.info('{}/{}: [{}|{}|{}|PID:{}|:{}]'.format(i_serie, len(df_series) - 1, *df_series.loc[i_serie,
                ['Study Date', 'Series Time', 'Modality', 'Patient ID', 'Series Instance UID']]))
    
    # create the query dataset
    query_ds = create_dataset_from_dataframe_row(df_series.loc[i_serie, :], 'IMAGE', incl=image_level_filters)
    # add some more filters
    query_ds.InstanceNumber = ['1', df_series.loc[i_serie, 'Number of Series Related Instances']]
    # display the Dataset
    logging.debug('Query Dataset:')
    for s in str(query_ds).split('\n'): logging.debug('    ' + s)

    # fetch the data (C-MOVE)
    df, _ = get_data(config, query_ds, to_fetch_params)
    df.reset_index(drop=True, inplace=True)
    
    # if no data is found, skip with error
    if len(df) == 0:
        logging.error('  ERROR, no data found.')
        continue
        """
        logging.error('  Problem, trying alternative indices')
        # try with a different index
        query_ds.InstanceNumber = ['2001', str(2000 + int(df_series.loc[i_serie, 'Number of Series Related Instances']))]
        # fetch the data (C-MOVE)
        df, _ = get_data(config, query_ds, to_fetch_params)
        df.reset_index(drop=True, inplace=True)
        # if no data is found, skip with error
        if len(df) == 0:
            logging.error('  ERROR, no data found.')
            continue
        """
    # if there is only one row, duplicate it
    elif len(df) == 1:
        df.loc[1, :] = df.loc[0, :]
    # if there are too many rows, skip with error
    elif len(df) > 2:
        logging.error('  ERROR, too many rows found ({}).'.format(len(df)))
        continue
       
    # copy the relevant parameters into the main DataFrame
    df_series.loc[i_serie, 'Acquisition Time Start'] = df.loc[0, 'AcquisitionTime'].split('.')[0]
    df_series.loc[i_serie, 'Acquisition Time End']   = df.loc[1, 'AcquisitionTime'].split('.')[0]
    df_series.loc[i_serie, 'Manufacturer Model Name']  = df.loc[0, 'ManufacturerModelName']
    df_series.loc[i_serie, 'Actual Frame Duration']  = df.loc[0, 'ActualFrameDuration']
    df_series.loc[i_serie, 'Number of Slices']  = df.loc[0, 'NumberOfSlices']
    
df_series

2019-10-24 16:22:05,772|INFO| 0/13: [20190806|080922|CT|PID:81334|:1.3.12.2.1107.5.1.4.10001.30000019080605520646600000009]
2019-10-24 16:22:07,095|INFO| 1/13: [20190806|081033|CT|PID:81334|:1.3.12.2.1107.5.1.4.10001.30000019080607103330600000000]
2019-10-24 16:22:07,954|INFO| 2/13: [20190806|081242|PT|PID:81334|:1.3.12.2.1107.5.1.4.10001.30000019080607124279600014406]
2019-10-24 16:22:08,827|INFO| 3/13: [20190806|081242|PT|PID:81334|:1.3.12.2.1107.5.1.4.10001.30000019080607124279600018501]
2019-10-24 16:22:09,685|INFO| 4/13: [20190806|081242|PT|PID:81334|:1.3.12.2.1107.5.1.4.10001.30000019080607124279600016387]
2019-10-24 16:22:10,517|INFO| 5/13: [20190806|081242|PT|PID:81334|:1.3.12.2.1107.5.1.4.10001.30000019080607124279600000664]
2019-10-24 16:22:11,331|INFO| 6/13: [20190806|081242|PT|PID:81334|:1.3.12.2.1107.5.1.4.10001.30000019080607124279600000265]
2019-10-24 16:22:12,158|INFO| 7/13: [20190806|081242|PT|PID:81334|:1.3.12.2.1107.5.1.4.10001.30000019080607124279600006081]
2019-10-

,Study Date,Series Date,Study Time,Series Time,Modality,Series Description,Patient ID,Study Instance UID,Series Instance UID,Number of Series Related Instances,Acquisition Time Start,Acquisition Time End,Manufacturer Model Name,Actual Frame Duration,Number of Slices
0,20190806,20190806,080313,080922,CT,Topogram 0.6 T80f,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080605520646600000009,1,080935,080935,Biograph64,NaN,NaN
1,20190806,20190806,080313,081033,CT,AC CT Cardiac 3.0 HD_FoV rest iMAR,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607103330600000000,132,081025,081030,Biograph64,NaN,NaN
2,20190806,20190806,080313,081242,PT,Rest PET CardioFreeze single 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600014406,1056,081455,081455,Biograph64_Vision 600,366000.0,132.0
3,20190806,20190806,080313,081242,PT,Rest PET CardioFreeze dual 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600018501,1056,081455,081455,Biograph64_Vision 600,366000.0,132.0
4,20190806,20190806,080313,081242,PT,Rest PET Cardiac gated 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600016387,1056,081455,081455,Biograph64_Vision 600,366000.0,132.0
5,20190806,20190806,080313,081242,PT,Rest PET Cardiac Static,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600000664,132,081455,081455,Biograph64_Vision 600,366000.0,132.0
6,20190806,20190806,080313,081242,PT,Rest PET Cardiac NAC Static,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600000265,132,081455,081455,Biograph64_Vision 600,366000.0,132.0
7,20190806,20190806,080313,081242,PT,Rest PET Cardiac Dynamic,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600006081,2772,081255,081901,Biograph64_Vision 600,8000.0,132.0
8,20190806,20190806,080313,083024,PT,Stress PET CardioFreezeDual 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600037001,1056,083237,083237,Biograph64_Vision 600,366000.0,132.0
9,20190806,20190806,080313,083024,PT,Stress PET Cardio Gated 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600034888,1056,083237,083238,Biograph64_Vision 600,366000.0,132.0


Trying to understand times
--

In [48]:
df_same_times = df_series[df_series['Acquisition Time Start'] == df_series['Acquisition Time End']]
ICD.display(df_same_times)
df_diff_times = df_series[df_series['Acquisition Time Start'] != df_series['Acquisition Time End']]
ICD.display(df_diff_times)

,Study Date,Series Date,Study Time,Series Time,Modality,Series Description,Patient ID,Study Instance UID,Series Instance UID,Number of Series Related Instances,Acquisition Time Start,Acquisition Time End,Manufacturer Model Name,Actual Frame Duration,Number of Slices
0,20190806,20190806,080313,080922,CT,Topogram 0.6 T80f,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080605520646600000009,1,080935,080935,Biograph64,NaN,NaN
2,20190806,20190806,080313,081242,PT,Rest PET CardioFreeze single 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600014406,1056,081455,081455,Biograph64_Vision 600,366000.0,132.0
3,20190806,20190806,080313,081242,PT,Rest PET CardioFreeze dual 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600018501,1056,081455,081455,Biograph64_Vision 600,366000.0,132.0
4,20190806,20190806,080313,081242,PT,Rest PET Cardiac gated 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600016387,1056,081455,081455,Biograph64_Vision 600,366000.0,132.0
5,20190806,20190806,080313,081242,PT,Rest PET Cardiac Static,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600000664,132,081455,081455,Biograph64_Vision 600,366000.0,132.0
6,20190806,20190806,080313,081242,PT,Rest PET Cardiac NAC Static,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600000265,132,081455,081455,Biograph64_Vision 600,366000.0,132.0
8,20190806,20190806,080313,083024,PT,Stress PET CardioFreezeDual 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600037001,1056,083237,083237,Biograph64_Vision 600,366000.0,132.0
10,20190806,20190806,080313,083024,PT,Stress PET CardioFreezeSingle 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600020885,1056,083237,083237,Biograph64_Vision 600,366000.0,132.0
11,20190806,20190806,080313,083024,PT,Stress PET Cardiac NAC Static,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600006610,132,083237,083237,Biograph64_Vision 600,366000.0,132.0
12,20190806,20190806,080313,083024,PT,Stress PET Cardiac Static,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600007405,132,083237,083237,Biograph64_Vision 600,366000.0,132.0


,Study Date,Series Date,Study Time,Series Time,Modality,Series Description,Patient ID,Study Instance UID,Series Instance UID,Number of Series Related Instances,Acquisition Time Start,Acquisition Time End,Manufacturer Model Name,Actual Frame Duration,Number of Slices
1,20190806,20190806,080313,081033,CT,AC CT Cardiac 3.0 HD_FoV rest iMAR,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607103330600000000,132,081025,081030,Biograph64,NaN,NaN
7,20190806,20190806,080313,081242,PT,Rest PET Cardiac Dynamic,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600006081,2772,081255,081901,Biograph64_Vision 600,8000.0,132.0
9,20190806,20190806,080313,083024,PT,Stress PET Cardio Gated 8 Gates,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600034888,1056,083237,083238,Biograph64_Vision 600,366000.0,132.0
13,20190806,20190806,080313,083024,PT,Stress PET Cardiac Dynamic,81334,1.2.826.0.1.3680043.2.146.2.20.81334.1900166323.0,1.3.12.2.1107.5.1.4.10001.30000019080607124279600012953,2772,083037,083643,Biograph64_Vision 600,8000.0,132.0


Creating time ranges
--

In [49]:
df_series_original = df_series.copy()

In [51]:
df_series = df_series_original.copy()

In [52]:
Range = namedtuple('Range', ['start', 'end'])
FMT = '%H%M%S'

df_series.drop_duplicates(['Acquisition Time Start', 'Acquisition Time End'], inplace=True)
df_series.sort_values('Acquisition Time Start', inplace=True)

inclusion_found = True
while inclusion_found:
    inclusion_found = False
    df_series.reset_index(drop=True, inplace=True)
    for i_serie in range(1, len(df_series)):
        curr_start = dt.strptime(df_series.iloc[i_serie]['Acquisition Time Start'], FMT)
        curr_end = dt.strptime(df_series.iloc[i_serie]['Acquisition Time End'], FMT)
        # check if we are in range of previous one
        prev_start = dt.strptime(df_series.iloc[i_serie - 1]['Acquisition Time Start'], FMT)
        prev_end = dt.strptime(df_series.iloc[i_serie - 1]['Acquisition Time End'], FMT)

        r1 = Range(start=curr_start, end=curr_end)
        r2 = Range(start=prev_start, end=prev_end)
        latest_start = max(r1.start, r2.start)
        earliest_end = min(r1.end, r2.end)
        delta = (earliest_end - latest_start).seconds
        overlap = max(0, delta)
        inclusion_found = overlap == 0
        
        logging.info('{:2}/{}: checking if {}-{} is included in {}-{}: overlap = {}'.format(i_serie, len(df_series) - 1,
            curr_start.strftime(FMT), curr_end.strftime(FMT), prev_start.strftime(FMT), prev_end.strftime(FMT), inclusion_found))
        
        if inclusion_found:
            df_series.drop(i_serie,inplace=True)
            break



2019-10-24 16:23:12,766|INFO|  1/6: checking if 081025-081030 is included in 080935-080935: overlap = False
2019-10-24 16:23:12,768|INFO|  2/6: checking if 081255-081901 is included in 081025-081030: overlap = False
2019-10-24 16:23:12,770|INFO|  3/6: checking if 081455-081455 is included in 081255-081901: overlap = True
2019-10-24 16:23:12,772|INFO|  1/5: checking if 081025-081030 is included in 080935-080935: overlap = False
2019-10-24 16:23:12,775|INFO|  2/5: checking if 081255-081901 is included in 081025-081030: overlap = False
2019-10-24 16:23:12,777|INFO|  3/5: checking if 083037-083643 is included in 081255-081901: overlap = False
2019-10-24 16:23:12,780|INFO|  4/5: checking if 083237-083237 is included in 083037-083643: overlap = True
2019-10-24 16:23:12,782|INFO|  1/4: checking if 081025-081030 is included in 080935-080935: overlap = False
2019-10-24 16:23:12,784|INFO|  2/4: checking if 081255-081901 is included in 081025-081030: overlap = False
2019-10-24 16:23:12,787|INFO| 

In [53]:
for i_serie in range(len(df_series)):
    if str(df_series.loc[i_serie, 'Acquisition Time Start']) == 'nan': continue
    start_stop_range = '{}-{}'.format(df_series.loc[i_serie, 'Acquisition Time Start'],
                                      df_series.loc[i_serie, 'Acquisition Time End'])
    df_series.loc[i_serie, 'Start Stop Range'] = start_stop_range  

df_series.loc[:, 'Start Stop Range'].unique()

array(['080935-080935', '081025-081030', '081255-081901', '083037-083643',
       '083237-083238'], dtype=object)

Refine the data set based on the "Institution Name" and the "Patient ID"
--

In [ ]:
accepted_inst_names = ['chuv', 'chuvlausanne', 'centrehospitalieruniversitairevaudois', 'petctchuv',
                       'medecinenucleairechuvlausanne']
df_all_series['Short Institution Name'] = df_all_series['Institution Name'].str.lower().apply(lambda s: str(s).replace(' ', ''))
df_series = df_all_series[df_all_series['Short Institution Name'].isin(accepted_inst_names)]
df_series = df_series[df_series['Patient ID'].str.match('^\d+$')].reset_index(drop=True)
df_series
count_inst_name = df_series.groupby('Institution Name')['Institution Name'].count()
count_inst_name['None'] = len(df_series[df_series['Institution Name'].isnull()])
count_inst_name

Go through each Series and get more info
--

In [ ]:
df_series_updated = df_series.copy()

# go through each Serie
#for i_serie in range(len(df_series)):
for i_serie in range(0, 20):
    
    logging.info('{}/{}: [{}|{}|{}|PID:{}|:{}]'.format(i_serie, len(df_series), *df_series.loc[i_serie,
                ['Study Date', 'Series Time', 'Modality', 'Patient ID', 'Series Instance UID']]))
    query_ds = Dataset()
    # set all the parameters for filtering
    for col in df_series.columns:
        col_name = col.title().replace(' ', '').replace('Uid', 'UID')
        value = df_series.loc[i_serie, col]
        # avoid NaNs
        if str(value) == 'nan': value = ''
        logging.debug('Setting "{}" ("{}") to "{}" (type: "{}")'.format(col, col_name, value, type(value)))
        setattr(query_ds, col_name, value)
        
    # set the specific filtering parameters
    query_ds.QueryRetrieveLevel = 'IMAGE'
    query_ds.InstanceNumber = ['1', df_series.loc[i_serie, 'Number of Series Related Instances']]
    # display the Dataset
    logging.debug('Query Dataset:')
    for s in str(query_ds).split('\n'): logging.debug('    ' + s)

    # fetch the data (C-MOVE)
    to_fetch_param_names = ['ManufacturerModelName', 'InstanceNumber', 'ContentTime',
                            'AcquisitionTime', 'ActualFrameDuration', 'NumberOfSlices', 
                           '0x07a31023', 'SOPClassUID' ]
    df, _ = get_data(config, query_ds, to_fetch_param_names)
    df.reset_index(drop=True, inplace=True)
    # if no data is found, skip with error
    if len(df) == 0:
        logging.error('  Problem, no data found.')
        continue
    # if there is only one row, duplicate it
    elif len(df) == 1:
        df.loc[1, :] = df.loc[0, :]
    # if there are too many rows, skip with error
    elif len(df) > 2:
        logging.error('  Problem, too many rows found ({}).'.format(len(df)))
        continue
       
    # copy the relevant parameters into the main DataFrame
    df_series_updated.loc[i_serie, 'Content Time Start']     = df.loc[0, 'ContentTime']
    df_series_updated.loc[i_serie, 'Content Time End']       = df.loc[1, 'ContentTime']
    df_series_updated.loc[i_serie, 'Acquisition Time Start'] = df.loc[0, 'AcquisitionTime']
    df_series_updated.loc[i_serie, 'Acquisition Time End']   = df.loc[1, 'AcquisitionTime']
    df_series_updated.loc[i_serie, 'Manufacturer Model Name']  = df.loc[0, 'ManufacturerModelName']
    df_series_updated.loc[i_serie, 'Actual Frame Duration']  = df.loc[0, 'ActualFrameDuration']
    df_series_updated.loc[i_serie, 'Number of Slices']  = df.loc[0, 'NumberOfSlices']
    df_series_updated.loc[i_serie, '0x07a31023']  = df.loc[0, '0x07a31023']
    df_series_updated.loc[i_serie, 'SOP Class UID']  = df.loc[0, 'SOPClassUID']
    
df_series_updated

In [ ]:
df_series_updated.loc[0:20, :]

Figure out why some data cannot be fetched

In [ ]:
df_series.loc[0]

In [ ]:
i_serie = 1

logging.info('{}/{}: [{}|{}|{}|PID:{}|:{}]'.format(i_serie, len(df_series), *df_series.loc[i_serie,
                ['Study Date', 'Series Time', 'Modality', 'Patient ID', 'Series Instance UID']]))
query_ds = Dataset()
# set all the parameters for filtering
exclude_columns = ['Number of Series Related Instances', 'Instance Number']
for col in df_series.columns:
    if col in exclude_columns: continue
    col_name = col.title().replace(' ', '').replace('Uid', 'UID')
    value = df_series.loc[i_serie, col]
    # avoid NaNs
    if str(value) == 'nan': value = ''
    logging.info('Setting "{}" ("{}") to "{}" (type: "{}")'.format(col, col_name, value, type(value)))
    setattr(query_ds, col_name, value)

# set the specific filtering parameters
query_ds.QueryRetrieveLevel = 'IMAGE'
query_ds.InstanceNumber = '1'
# display the Dataset
logging.info('Query Dataset:')
for s in str(query_ds).split('\n'): logging.info('    ' + s)

# fetch the data (C-MOVE)
to_fetch_param_names = ['ManufacturerModelName', 'InstanceNumber', 'ContentTime',
                        'AcquisitionTime', 'ActualFrameDuration']
df, datasets = get_data(config, query_ds, to_fetch_param_names)
df

In [ ]:
datasets

Get (C-MOVE) a PT series first and last frame

In [ ]:
# Create out identifier (query) dataset
query_ds = Dataset()
query_ds.QueryRetrieveLevel = 'IMAGE'
query_ds.SeriesInstanceUID = '1.3.12.2.1107.5.6.1.2013.31330119100111184984100000023'
query_ds.PatientID = '69168'
query_ds.StudyDate  = '20191001'
query_ds.Modality = 'PT'
query_ds.InstanceNumber = ['1', '387']

df = get_data(config, query_ds, ['PatientID', 'ManufacturerModelName', 'InstanceNumber', 'StudyTime',
                                 'SeriesTime', 'ContentTime', 'AcquisitionTime'])
df

In [ ]:
# Create out identifier (query) dataset
query_ds = Dataset()
query_ds.QueryRetrieveLevel = 'IMAGE'
query_ds.SeriesInstanceUID = '1.3.12.2.1107.5.1.4.10001.30000019100107233748400041397'
query_ds.PatientID = '1084649'
query_ds.StudyDate  = '20191001'
query_ds.Modality = 'PT'

query_ds.InstanceNumber = ['1', '1000', '2772']

df = get_data(config, query_ds, ['PatientID', 'ManufacturerModelName', 'InstanceNumber', 'StudyTime',
                                 'SeriesTime', 'ContentTime', 'AcquisitionTime'])
df

Get all the series corresponding to the query Dataset's filter [date = 2019-10-01, study time = between 8am and 9am]

In [ ]:
query_ds = Dataset()

# set the filtering parameters
query_ds.QueryRetrieveLevel = 'SERIES'
query_ds.StudyDate  = '20191001'
query_ds.SeriesDate = '20191001'
query_ds.StudyTime  = '080000-090000'
query_ds.SeriesTime = '080000-090000'

# parameters to fetch
query_ds.NumberOfSeriesRelatedInstances = ''
query_ds.SeriesInstanceUID = ''
query_ds.Modality = ''
query_ds.PatientID = ''
query_ds.StudyID = ''

logging.info('Query Dataset:')
for s in str(query_ds).split('\n'): logging.info('    ' + s)

df_series_all = find_data(config, query_ds)
df_series_all

Get a list of Series having more than a few frames

In [ ]:
df_series = df_series_all[df_series_all['Number of Series Related Instances'].astype(int) > 2][0:3]
df_series

Get time differences by querying the ContentTime on the IMAGE level

In [ ]:
count_modality = df_series.groupby('Modality')['Modality'].count()
logging.info('Number of series per modality: NM = {NM} | PT = {PT}'.format(**count_modality))
n_instances_groupby_modality = df_series.groupby('Modality')['Number of Series Related Instances']
logging.info('Number of Instances [min]:  NM = {NM:6.1f} | PT = {PT:6.1f}'.format(**n_instances_groupby_modality.min()))
logging.info('Number of Instances [avg]:  NM = {NM:6.1f} | PT = {PT:6.1f}'.format(**n_instances_groupby_modality.mean()))
logging.info('Number of Instances [med]:  NM = {NM:6.1f} | PT = {PT:6.1f}'.format(**n_instances_groupby_modality.median()))
logging.info('Number of Instances [max]:  NM = {NM:6.1f} | PT = {PT:6.1f}'.format(**n_instances_groupby_modality.max()))


In [ ]:
time_format = '%H%M%S'

for i in df_series.index:
    
    series_UID = df_series.loc[i, 'Series Instance UID']
    last_frame = df_series.loc[i, 'Number of Series Related Instances']
    
    logging.info('Fetching info for index {}: "{}"'.format(i, series_UID))
    
    query_ds = Dataset()
    query_ds.QueryRetrieveLevel = 'IMAGE'
    query_ds.InstanceNumber = ['1', last_frame]
    query_ds.SeriesInstanceUID = series_UID
    query_ds.ContentTime = ''
    
    df = retrieve_data(config, query_ds)
    
    start_time = dt.strptime(df.loc[0, 'Content Time'], time_format)
    end_time = dt.strptime(df.loc[1, 'Content Time'], time_format)
    delta_sec = (end_time - start_time).seconds

    logging.info('Series total time: end [{:%H:%M:%S}] - [{:%H:%M:%S}] = {} seconds'.format(end_time, start_time, delta_sec))

In [ ]:
df_series_pt = \
    df_series_all[
        (df_series_all.Modality == 'PT')
      & (df_series_all['Number of Series Related Instances'].astype(int) > 1)
    ][0:10]
ICD.display(df_series_pt)
print(df_series_pt.loc[556, 'Series Instance UID'])
print(df_series_pt.loc[556, 'Patient ID'])

In [ ]:
time_format = '%H%M%S'

for i in df_series_pt.index:
    
    series_UID = df_series_pt.loc[i, 'Series Instance UID']
    last_frame = df_series_pt.loc[i, 'Number of Series Related Instances']
    mid_frame = str(int(int(last_frame) / 2))
    
    logging.info('Fetching info for index {}: "{}"'.format(i, series_UID))
    
    query_ds = Dataset()
    query_ds.QueryRetrieveLevel = 'IMAGE'
    query_ds.InstanceNumber = ['1', mid_frame, last_frame]
    query_ds.SeriesInstanceUID = series_UID
    query_ds.StudyTime = ''
    query_ds.SeriesTime = ''
    query_ds.ContentTime = ''
    query_ds.NumberOfSeriesRelatedInstances = ''
    
    df = retrieve_data(config, query_ds)
    if len(df) > 0:
        ICD.display(df)

In [ ]:
df.loc[0, 'Content Time'].split('.')[0]

Get all PT images with InstanceNumber 1 for a specific day (20191001)

In [ ]:
query_ds = Dataset()

# set the filtering parameters
query_ds.QueryRetrieveLevel = 'IMAGE'
query_ds.StudyDate  = '20191001'
query_ds.InstanceNumber = '1'
query_ds.Modality = 'PT'

# parameters to fetch
query_ds.StudyTime = ''
query_ds.SeriesTime = ''
query_ds.ContentTime = ''
query_ds.NumberOfSeriesRelatedInstances = ''
query_ds.SeriesInstanceUID = ''

logging.info('Query Dataset:')
for s in str(query_ds).split('\n'): logging.info('    ' + s)

df_PT_IN1_20191001_all = retrieve_data(config, query_ds)
df_PT_IN1_20191001_all

Get the content time of the last frame

In [ ]:
df_PT_IN1_20191001_copy = df_PT_IN1_20191001_all.copy()
for i in df_PT_IN1_20191001_copy.index:
    
    series_UID = df_PT_IN1_20191001_copy.loc[i, 'Series Instance UID']
    last_frame = df_PT_IN1_20191001_copy.loc[i, 'Number of Series Related Instances']
    mid_frame = str(int(int(last_frame) / 2))
    
    logging.info('Fetching info for index {}: "{}"'.format(i, series_UID))
    
    query_ds = Dataset()
    query_ds.QueryRetrieveLevel = 'IMAGE'
    query_ds.InstanceNumber = last_frame
    query_ds.SeriesInstanceUID = series_UID
    query_ds.ContentTime = ''
    
    df = retrieve_data(config, query_ds)
    if len(df) > 0:
        df_PT_IN1_20191001_copy.loc[i, 'Last Content Time'] = df.loc[0, 'Content Time']
    
    query_ds = Dataset()
    query_ds.QueryRetrieveLevel = 'IMAGE'
    query_ds.InstanceNumber = mid_frame
    query_ds.SeriesInstanceUID = series_UID
    query_ds.ContentTime = ''
    
    df = retrieve_data(config, query_ds)
    if len(df) > 0:
        df_PT_IN1_20191001_copy.loc[i, 'Mid Content Time'] = df.loc[0, 'Content Time']

In [ ]:
df_PT_IN1_20191001_copy[['Content Time', 'Mid Content Time', 'Last Content Time', 'Instance Number', 'Number of Series Related Instances']]

In [ ]:
s = df_PT_IN1_20191001_copy.loc[0:39, 'Content Time'].apply(lambda s: s.split('.')[1]).astype(int)
e = df_PT_IN1_20191001_copy.loc[0:39, 'Last Content Time'].apply(lambda s: s.split('.')[1]).astype(int)
n = df_PT_IN1_20191001_copy.loc[0:39, 'Number of Series Related Instances'].astype(int)
d = e - s
import pandas as pd
a = pd.DataFrame([s, e, n, d, d/n]).transpose()
a.columns = ['s', 'e', 'n', 'd', 'd/n']
a[a['n'] > 400]

Try to use MOVE instead of FIND

In [ ]:
df_series.loc[0]['Series Instance UID']

In [ ]:
ds = Dataset()
ds.QueryRetrieveLevel = 'IMAGE'
ds.SeriesInstanceUID = '1.2.840.113619.2.80.3834766814.478.1569931317.104.4.1'
ds.StudyInstanceUID = '1.2.826.0.1.3680043.2.146.2.20.2560235.1900228608.0'
ds.PatientID = '2560235'
ds.StudyID = '40464'
ds.StudyDate  = '20191001'
ds.Modality = 'CT'
ds.InstanceNumber = '1'

df = retrieve_data(config, ds)
df

In [ ]:
from pydicom.dataset import Dataset

from pynetdicom import AE, evt, StoragePresentationContexts
from pynetdicom.sop_class import PatientRootQueryRetrieveInformationModelMove

datasets = []
def handle_store(event):
    """Handle a C-STORE service request"""
    print('HANDLE STORE')
    datasets.append(event.dataset)
    # Ignore the request and return Success
    return 0x0000

handlers = [(evt.EVT_C_STORE, handle_store)]

# Initialise the Application Entity
ae = AE(ae_title = config['PACS']['ae_title'])

# Add a requested presentation context
ae.add_requested_context(PatientRootQueryRetrieveInformationModelMove)

# Add the Storage SCP's supported presentation contexts
ae.supported_contexts = StoragePresentationContexts

# Start our Storage SCP in non-blocking mode, listening on port 11120
ae.ae_title = config['PACS']['ae_title']
scp = ae.start_server((config['PACS']['local_host'], config['PACS'].getint('local_port')), block=False, evt_handlers=handlers)

# Create out identifier (query) dataset
ds = Dataset()
ds.QueryRetrieveLevel = 'IMAGE'
ds.SeriesInstanceUID = '1.2.840.113619.2.80.3834766814.478.1569931317.104.4.1'
ds.StudyInstanceUID = '1.2.826.0.1.3680043.2.146.2.20.2560235.1900228608.0'
ds.PatientID = '2560235'
ds.StudyID = '40464'
ds.StudyDate  = '20191001'
ds.Modality = 'CT'
ds.InstanceNumber = ['1', '100', '200', '289', '298', '300']

# Associate with peer AE at IP 127.0.0.1 and port 11112
assoc = ae.associate(config['PACS']['host'], config['PACS'].getint('port'), ae_title=config['PACS']['ae_called_title'])

if assoc.is_established:
    # Use the C-MOVE service to send the identifier
    responses = assoc.send_c_move(ds, config['PACS']['ae_title'], PatientRootQueryRetrieveInformationModelMove)

    for (status, identifier) in responses:
        if status:
            print('C-MOVE query status: 0x{0:04x}'.format(status.Status))

            # If the status is 'Pending' then `identifier` is the C-MOVE response
            if status.Status in (0xFF00, 0xFF01):
                print(identifier)
        else:
            print('Connection timed out, was aborted or received invalid response')

    # Release the association
    assoc.release()
else:
    print('Association rejected, aborted or never connected')

# Stop our Storage SCP
scp.shutdown()


In [ ]:
[ds.ContentTime for ds in datasets]

In [ ]:
datasets[0].ManufacturerModelName

In [ ]:
# Create out identifier (query) dataset
query_ds = Dataset()
query_ds.QueryRetrieveLevel = 'IMAGE'
query_ds.SeriesInstanceUID = '1.2.840.113619.2.80.3834766814.478.1569931317.104.4.1'
query_ds.StudyInstanceUID = '1.2.826.0.1.3680043.2.146.2.20.2560235.1900228608.0'
query_ds.PatientID = '2560235'
query_ds.StudyID = '40464'
query_ds.StudyDate  = '20191001'
query_ds.Modality = 'CT'
query_ds.InstanceNumber = ['1', '100', '131', '132']

df = find_data(config, query_ds)
df